In [100]:
import pandas as pd 

df_domain = pd.read_csv('domain_page_rank_2022.csv')

# count the number of nan in the page_rank column
df_domain.page_rank.isnull().sum() 

20209

In [101]:
df_domain.head()
len(df_domain)

135427

In [102]:
df_host = pd.read_csv('host_name_page_rank_2022.csv')
df_host.page_rank.isnull().sum()

52903

In [103]:
df_host.head()

,hostnames,country,alpha2,alpha3,hostnames_reverse,domain,page_rank
0,services.iacad.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.iacad.services,ae.gov.iacad.services,15860617.0
1,eservices.nmc.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.nmc.eservices,ae.gov.nmc.eservices,71243866.0
2,www.ajmanport.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.ajmanport.www,ae.gov.ajmanport,NaN
3,dc.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.dc,ae.gov.dc,741114.0
4,sdg.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.sdg,ae.gov.sdg,15288081.0


In [104]:
# how to merge two dataframes, fill out the missing values in the page_rank column
df_domain = df_domain.merge(df_host, on='hostnames', how='left')
# we fill out the missing values in the page_rank_x column with the values in the page_rank_y column
df_domain.page_rank_x.fillna(df_domain.page_rank_y, inplace=True)
df_domain.drop('page_rank_y', axis=1, inplace=True)
df_domain.rename(columns={'page_rank_x':'page_rank'}, inplace=True)
df_domain.page_rank.isnull().sum()

19735

In [105]:
# drop the columns that end with _y
df_domain.drop(df_domain.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
# rename the columns that end with _x
df_domain.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
df_domain.head()


,hostnames,country,alpha2,alpha3,hostnames_reverse,domain,page_rank
0,www.govern.ad,Andorra,AD,AND,ad.govern.www,ad.govern,170739.0
1,www.tramits.ad,Andorra,AD,AND,ad.tramits.www,ad.tramits,4127309.0
2,www.protecciocivil.ad,Andorra,AD,AND,ad.protecciocivil.www,ad.protecciocivil,2244704.0
3,ia.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.ia,ae.gov.ia,759105.0
4,wsn.hw.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.hw.wsn,ae.gov.hw.wsn,26547208.0


In [107]:
# clearn the dataset
df_domain = df_domain[df_domain['alpha2'].notna()]
print(len(df_domain))

# remove the noise dataset for alpha2_code
df_domain = df_domain[df_domain['alpha2'].str.len() == 2]
len(df_domain)

135392


114998

In [108]:
# write the file to csv
df_domain.to_csv('government_websites_page_rank_2022.csv', index=False)


In [109]:
print(len(df_domain))   

114998


In [110]:
# group by country and select the top 500 based on page_rank
# df_domain['page_rank'].fillna(0, inplace=True)
df_group_country = df_domain.groupby('country').apply(lambda x: x.nsmallest(500, 'page_rank')).reset_index(drop=True)
print(df_group_country.head())
print(len(df_group_country))

              hostnames      country alpha2 alpha3     hostnames_reverse  \
0      president.gov.af  Afghanistan     AF    AFG      af.gov.president   
1  www.president.gov.af  Afghanistan     AF    AFG  af.gov.president.www   
2            mfa.gov.af  Afghanistan     AF    AFG            af.gov.mfa   
3        www.mfa.gov.af  Afghanistan     AF    AFG        af.gov.mfa.www   
4        www.mof.gov.af  Afghanistan     AF    AFG        af.gov.mof.www   

             domain  page_rank  
0  af.gov.president   303963.0  
1  af.gov.president   303963.0  
2        af.gov.mfa   391095.0  
3        af.gov.mfa   391095.0  
4        af.gov.mof   525054.0  
30845


In [115]:
# write file to csv
df_group_country.to_csv('government_websites_page_rank_2022_top_500.csv', index=False)
df_group_country.page_rank.isnull().sum()

1236

In [111]:
# count the numer of domains per country
df_group_country.country.value_counts()

Thailand             500
Ukraine              500
South Korea          500
Pakistan             500
Colombia             500
                    ... 
Equatorial Guinea      3
Niger                  3
Togo                   2
Guinea-Bissau          2
Grenada                1
Name: country, Length: 197, dtype: int64

In [112]:
df_country_clean = df_group_country['alpha3']

In [83]:
# rm the noise the country

0                    af.gov.afghanpost.webmail
1                     af.gov.superemecourt.www
2                            af.gov.dgsd.ncmis
3                           af.gov.pcu-mew.mis
4                             af.gov.mcit.jobs
                         ...                  
30840                   zw.gov.testdomain4.www
30841    zw.gov.postelectionviolencecommission
30842                         zw.gov.women.www
30843                      zw.gov.auditgen.www
30844                           zw.gov.ddf.www
Name: hostnames_reverse, Length: 30845, dtype: object

In [84]:
len(df_group_country)

30845

In [113]:
# filter the country that has great than 500 domains

df_group_country.country.value_counts().reindex().to_csv('country_count.csv')